In [4]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [5]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

--2019-11-07 12:28:05--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.109.59
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.109.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10367709 (9.9M) [application/json]
Saving to: ‘./pw-data/201701scripts_sample.json.gz’

     0K .......... .......... .......... .......... ..........  0% 25.1M 0s
    50K .......... .......... .......... .......... ..........  0% 30.7M 0s
   100K .......... .......... .......... .......... ..........  1% 29.1M 0s
   150K .......... .......... .......... .......... ..........  1% 41.0M 0s
   200K .......... .......... .......... .......... ..........  2% 39.8M 0s
   250K .......... .......... .......... .......... ..........  2%  321M 0s
   300K .......... .......... .......... .......... ..........  3% 51.2M 0s
   35

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [2]:
import gzip
import simplejson as json

In [3]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]
print("")

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [12]:
i = [key for key in scripts[0].keys()]
i

['bnf_code', 'items', 'practice', 'bnf_name', 'nic', 'act_cost', 'quantity']

In [1]:
"""Statistics Method"""
import math
def mean(data):
    n = len(data)
    return sum(data)/n

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def sd(data):
    return math.sqrt(_ss(data)/(len(data)-1))

def mode(data):
    return max(set(data), key = data.count)

def median(data):
    sort = sorted(data)
    l = len(sort)
    mid = (l - 1) // 2
    if (l % 2):
        return sort[mid]
    else:
        return (sort[mid] + sort[mid + 1])/2

def quartile(data):
    sort = sorted(data)
    l = len(sort)
    mid = (l + 1) // 2
    if mid == (l + 1) // 2:
#        print(sort[:mid],sort[mid:])
        return median(sort[:mid]),median(sort[mid:])
    else :
#        print(sort[:mid],sort[mid+1:])
        return median(sort[:mid]),median(sort[mid + 1:])
       

sample_data = [1, 12, 3,11, 15, 5, 10, 2, 1,6,8]
print(sorted(sample_data))

quartile(sample_data)

[1, 1, 2, 3, 5, 6, 8, 10, 11, 12, 15]


(2.5, 11)

In [ ]:
def get_keys(lst):
    return [key for key in lst[0].keys()] 

def make_data_dic(lst):
    data_dic = {}
    data_keys = get_keys(lst)
    
    """Make EMPTY LIST of Data Dictionary of each Keys"""
    for key in data_keys:
        data_dic[key] = []
    """Add All the Data to Specific key by opening each entry in First Loop & add on data_dic in next loop"""
    for entry in lst:
        for key in data_keys:
            data_dic[key].append(entry[key]) 
    return data_dic

def get_data(dic,key):
    return dic[key]

In [17]:
def get_keys(lst):
    return [key for key in lst[0].keys()] 

def make_data_dic(lst):
    data_dic = {}
    data_keys = get_keys(lst)
    
    """Make EMPTY LIST of Data Dictionary of each Keys"""
    for key in data_keys:
        data_dic[key] = []
    """Add All the Data to Specific key by opening each entry in First Loop & add on data_dic in next loop"""
    for entry in lst:
        for key in data_keys:
            data_dic[key].append(entry[key]) 
    return data_dic

def get_data(dic,key):
    return dic[key]

def _index_list(item, my_list):
    """Return index list for a given key on list"""
    return [index for index, event in enumerate(my_list) if event == item ]

def _index_to_value(index_list,my_list):
    return [my_list[i] for i in index_list]

def _group_list_by(list_by,list_for,key_by):
    """Input = Two list and Key, Return second list """
    group_dic = {}
    unique_items = sorted(_unique_entry(list_by))[:100]
    for item in unique_items:
        index_list = _index_list(item, list_by)
#       print(item,"--->",_index_to_value(index_list,list_for))
        group_dic[item] = list(set(_index_to_value(index_list,list_for)))
    return group_dic

def _group_sum_by(list_by,list_for,key_by):
    """Input = Two list and Key, Return second list """
    group_dic = {}
    unique_items = sorted(_unique_entry(list_by))[:100]
    for item in unique_items:
        index_list = _index_list(item, list_by)
#       print(item,"--->",_index_to_value(index_list,list_for))
        group_dic[item] = sum(_index_to_value(index_list,list_for))
    return group_dic

def _unique_entry(my_list):
    return sorted(list(set(my_list)))

def groupby(key_by, key_for,my_list):
    data_dict = make_data_dic(my_list)
    return _group_list_by(data_dict[key_by],data_dict[key_for],key_by)

In [ ]:
def describe(key):
    key_data = scripts_list[key]
    total = sum(key_data)
    m = mean(key_data)
    md = median(key_data)
    sds = sd(key_data)
    first, third = quartile(key_data)
    return total, m ,sds,first,md,third

describe("items")

In [ ]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [124]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [22]:
bnf_names_list = scripts_list['bnf_name']
bnf_names = set(bnf_names_list)
assert(len(bnf_names) == 5619)

NameError: name 'scripts_list' is not defined

In [17]:
max_value(group)

NameError: name 'max_value' is not defined

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [ ]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    # INSERT ...

In [ ]:
groups = {name:  for name in bnf_names}

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [27]:
def _index_list(item, my_list):
    """Return index list for a given key on list"""
    return [index for index, event in enumerate(my_list) if event == item ]

def _index_to_value(index_list,my_list):
    return [my_list[i] for i in index_list]

def _group_list_by(list_by,list_for,key_by):
    """Input = Two list and Key, Return second list """
    group_dic = {}
    unique_items = sorted(_unique_entry(list_by))[:100]
    for item in unique_items:
        index_list = _index_list(item, list_by)
#       print(item,"--->",_index_to_value(index_list,list_for))
        group_dic[item] = list(set(_index_to_value(index_list,list_for)))
    return group_dic

def _group_sum_by(list_by,list_for,key_by):
    """Input = Two list and Key, Return second list """
    group_dic = {}
    unique_items = sorted(_unique_entry(list_by))[:100]
    for item in unique_items:
        index_list = _index_list(item, list_by)
#       print(item,"--->",_index_to_value(index_list,list_for))
        group_dic[item] = sorted(_index_to_value(index_list,list_for))[0]
    return group_dic

def _unique_entry(my_list):
    return sorted(list(set(my_list)))

def groupby(key_by, key_for,my_list):
    data_dict = make_data_dic(my_list)
    return _group_list_by(data_dict[key_by],data_dict[key_for],key_by)

code_by_post = groupby('post_code','code',practices)
#practice_postal = {v: k for k, v in code_by_post.items()}
joined = script
code_by_post

In [32]:
scripts_list = make_data_dic(scripts)sample = scripts[:1000]

In [10]:
def max_key(my_dict):
    key = max(my_dict.keys(), key = (lambda k: my_dict[k]))
    value = my_dict[key]
    return key, value

max_item = [max_key(bnf_name_by_items)]
print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [12]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [ ]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = ...

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

In [14]:
code_by_post = groupby('code','post_code',practices)

In [16]:
code_by_post['N81013']

KeyError: 'N81013'

In [15]:
joined = scripts[:]
for script in joined:
    script['post_code'] = code_by_post[script['practice']]

KeyError: 'N81013'

In [155]:
practice_postal = {v: k for k, v in code_by_post.items()}

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [ ]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = ...
    else:
        practice_postal[practice['code']] = ...

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [28]:
assert code_by_post['K82019'] == 'HP21 8TR'

KeyError: 'K82019'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [23]:
assert practice_postal['K82019'] == 'HP21 8TR'

NameError: name 'practice_postal' is not defined

Now we can join `practice_postal` to `scripts`.

In [34]:
joined = scripts[:]
for script in joined:
    script['post_code'] = [script['practice']]

In [24]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [ ]:
items_by_post = ...

In [27]:
postal_totals = groupby('post_code','items',scripts)

TypeError: 'int' object is not subscriptable

In [80]:
len(postal)

132

In [173]:
postal = list(postal_totals.items())


[('B11 4BW', 20673),
 ('B18 7AL', 19001),
 ('B21 9RY', 29103),
 ('B23 6DJ', 24859),
 ('B70 7AW', 36531),
 ('BB11 2DL', 34356),
 ('BB2 1AX', 28254),
 ('BB3 1PY', 54514),
 ('BB4 5SL', 29388),
 ('BB7 2JG', 44585),
 ('BB8 0JZ', 54380),
 ('BB9 7SR', 38224),
 ('BD3 8QH', 21010),
 ('BH18 8EE', 39413),
 ('BH23 3AF', 32545),
 ('BL1 8TU', 26132),
 ('BL3 5HP', 27147),
 ('BL9 0NJ', 32062),
 ('BL9 0SN', 35275),
 ('CB9 8HF', 51337),
 ('CH1 4DS', 34915),
 ('CH65 6TG', 25090),
 ('CT11 8AD', 44358),
 ('CV1 4FS', 37210),
 ('CW1 3AW', 64104),
 ('CW5 5NX', 38797),
 ('CW7 1AT', 43164),
 ('DA1 2HA', 26075),
 ('DA11 8BZ', 24090),
 ('DN22 7XF', 43091),
 ('DN34 4GB', 48013),
 ('FY2 0JG', 69118),
 ('FY4 1TJ', 62886),
 ('FY5 2TZ', 44258),
 ('FY7 8GU', 34473),
 ('GL1 3PX', 38120),
 ('GL50 4DP', 74822),
 ('GU9 9QS', 32131),
 ('HA0 4UZ', 22755),
 ('HA3 7LT', 32113),
 ('HG1 5AR', 32684),
 ('HU7 4DW', 49107),
 ('KT14 6DH', 26758),
 ('KT6 6EZ', 38975),
 ('L31 0DJ', 32065),
 ('L36 7XY', 22965),
 ('L5 0QW', 24676),
 ('L

In [174]:
#postal_totals = [('B11 4BW', 20673)] * 100
grader.score.pw__postal_totals(postal)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [12]:
post_code__bnf_name = groupby('post_code','bnf_name',scripts)

KeyError: 'post_code'

In [80]:
def make_tuple(dicts):
    post_bnf = []
    for k in dicts.keys():
        l = [(k,i) for i in dicts[k]]
        post_bnf.extend(l)
    return post_bnf
 

In [10]:
post_code__items = groupby('post_code','items',scripts)

NameError: name 'make_data_dic' is not defined

In [81]:
post_code__bnf_name_tuple = make_tuple(post_code__bnf_name)
post_code__items_tuple = make_tuple(post_code__items)

In [ ]:
post_code__items_tuple[:10]

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [ ]:
total_items = ...
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [ ]:
max_item_by_post = ...

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [49]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [50]:
grader.score.pw__items_by_region(items_by_region)

Your score:  0.01


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*